In [1]:
import json
import pandas as pd
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from flask import Flask,request,jsonify
from flask_cors import CORS


In [2]:
with open("C:/Users/DELL/Desktop/meta.json",encoding='utf-8') as df:
    meta = json.load(df)
Meta = pd.DataFrame(meta['meta'])
Meta=Meta.dropna()

Meta['brand']=Meta['brand'].str.upper()
Meta['category']=Meta['category'].str.upper()
Meta['subcategory']=Meta['subcategory'].str.upper()
Meta['name']=Meta['name'].str.upper()

Meta = Meta.replace(',','', regex=True)


In [3]:
with open("C:/Users/DELL/Desktop/events.json",encoding='utf-8') as df2:
    Event_Data = json.load(df2)
Events = pd.DataFrame(Event_Data['events'])
Events=Events.dropna()

In [4]:
Meta = Meta.replace(',','', regex=True)

In [5]:
Events.head()

,event,sessionid,eventtime,price,productid
0,cart,a0655eee-1267-4820-af21-ad8ac068ff7a,2020-06-01T08:59:16.406Z,14.48,HBV00000NVZE8
1,cart,d2ea7bd3-9235-4a9f-a9ea-d7f296e71318,2020-06-01T08:59:46.580Z,49.90,HBV00000U2B18
2,cart,5e594788-78a0-44dd-8e66-37022d48f691,2020-06-01T08:59:33.308Z,1.99,OFIS3101-080
3,cart,fdfeb652-22fa-4153-b9b5-4dfa0dcaffdf,2020-06-01T08:59:31.911Z,2.25,HBV00000NVZBW
4,cart,9e9d4f7e-898c-40fb-aae9-256c40779933,2020-06-01T08:59:33.888Z,9.95,HBV00000NE0T4


In [6]:
Events.isnull().any()

event        False
sessionid    False
eventtime    False
price        False
productid    False
dtype: bool

In [7]:
def PhraseData(data):
        data_phrase = data.drop(columns=['productid'])
        data_phrase['phrase'] = data_phrase[data_phrase.columns[0:4]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
        data_phrase= data_phrase.drop(columns=['name','subcategory','brand','category'])
       
        return data_phrase

In [8]:
c=PhraseData(Meta)
c.head()

,phrase
0,"PALETTE,KIŞISEL BAKIM,SAÇ BAKIMI,PALETTE KALIC..."
1,"BEST,PET SHOP,KEDI,BEST PET JÖLE İÇINDE PARÇA ..."
2,"TARIM KREDI,TEMEL GIDA,BAKLIYAT PIRINÇ MAKARNA..."
3,"NAMET,ET BALIK ŞARKÜTERI,ŞARKÜTERI,NAMET FISTI..."
4,"MURATBEY,KAHVALTILIK VE SÜT,PEYNIR,MURATBEY BU..."


In [9]:
def TransformedData(data_phrase, data):
        object_count = CountVectorizer()
        count_matrix = object_count.fit_transform(data['name'])

        tfidf = TfidfVectorizer()
        tfidf_matrix = tfidf.fit_transform(data_phrase['phrase'])

        phrase_sparse = sp.hstack([count_matrix, tfidf_matrix], format='csr')
        cosine_sim = cosine_similarity(phrase_sparse, phrase_sparse)
        
        return cosine_sim

In [10]:
t=TransformedData(c,Meta)
print(t)

[[1.         0.         0.         ... 0.00195029 0.00184491 0.00146839]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.00195029 0.         0.         ... 1.         0.21658714 0.01102765]
 [0.00184491 0.         0.         ... 0.21658714 1.         0.00868691]
 [0.00146839 0.         0.         ... 0.01102765 0.00868691 1.        ]]


In [11]:
def FindProductsWithRelatedSessionId(sessionId,dataFrame):

    itemsList = []
    CardItemsId = []
    values = dataFrame.loc[dataFrame["sessionid"] == sessionId]
    itemsList.append(list(values["productid"]))
    count = 0
    for i in range(len(itemsList[0])):
        CardItemsId.append(itemsList[0][i])
        count = count + 1

    ItemsCount = count
    print(CardItemsId)
    return CardItemsId
    

In [12]:
cardsitem=FindProductsWithRelatedSessionId('f66b1455-d46c-45f5-b1c9-8af1d2fb12f3',Events)

['HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF']


In [13]:
for i in range(len(cardsitem)):
    itemsList = []
    values = Meta.loc[Meta["productid"] == cardsitem[i]]
    itemsList.append(list(values["name"]))
    print(values)
                           

         productid    brand   category      subcategory  \
920  HBV00000UIYEF  ÇAMLICA  İÇECEKLER  GAZLI İÇECEKLER   

                               name  
920  ÇAMLICA LIGHT GAZOZ 200 ML CAM  
         productid    brand   category      subcategory  \
920  HBV00000UIYEF  ÇAMLICA  İÇECEKLER  GAZLI İÇECEKLER   

                               name  
920  ÇAMLICA LIGHT GAZOZ 200 ML CAM  
         productid    brand   category      subcategory  \
920  HBV00000UIYEF  ÇAMLICA  İÇECEKLER  GAZLI İÇECEKLER   

                               name  
920  ÇAMLICA LIGHT GAZOZ 200 ML CAM  
         productid    brand   category      subcategory  \
920  HBV00000UIYEF  ÇAMLICA  İÇECEKLER  GAZLI İÇECEKLER   

                               name  
920  ÇAMLICA LIGHT GAZOZ 200 ML CAM  
         productid    brand   category      subcategory  \
920  HBV00000UIYEF  ÇAMLICA  İÇECEKLER  GAZLI İÇECEKLER   

                               name  
920  ÇAMLICA LIGHT GAZOZ 200 ML CAM  
         productid    bra

In [14]:
def RecommendedProducts(cardlist, data, combine, transform):
        
        newlist = list(dict.fromkeys(cardlist))
        simScoresForAllProducts =  list()
        for i in range(len(newlist)):
            
            try:
                indices = pd.Series(data.index, index = data['productid'])
                #print(indices['HBV00000NVZE8'])
                # print(cardlist[i])
                index = indices[newlist[i]]

                sim_scores = list(enumerate(transform[index]))
                sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
                sim_scores = sim_scores[1:11]

                for i in range(len(sim_scores)):
                    simScoresForAllProducts.append(sim_scores[i])
            except:
                print("Product has no brand")

        
        simScoresForAllProducts.sort()
        print(simScoresForAllProducts)
        
        product_indices = [i[0] for i in simScoresForAllProducts[1:11]]

        product_id = data['productid'].iloc[product_indices]
        product_brand = data['brand'].iloc[product_indices]
        product_category = data['category'].iloc[product_indices]
        product_subcategory = data['subcategory'].iloc[product_indices]
        product_name = data['name'].iloc[product_indices]

        recommendation_data = pd.DataFrame(columns=['productid','brand', 'category','subcategory','name'])

        recommendation_data['productid'] = product_id
        recommendation_data['brand'] = product_brand
        recommendation_data['category'] = product_category
        recommendation_data['subcategory'] = product_subcategory
        recommendation_data['name'] = product_name

        return recommendation_data

In [15]:
recommendations=RecommendedProducts(cardsitem, Meta, c, t)
recommendation_dict=recommendations.to_dict("records")
print(recommendation_dict)

[(253, 0.5371105831491507), (341, 0.5890969625071492), (523, 0.7691529366250116), (643, 0.7688705242838757), (835, 0.5634414865778319), (969, 0.5371105831491507), (2077, 0.6647009616560932), (2728, 0.5767100799945043), (6173, 0.6974603618736324), (6350, 0.664531135506103)]
[{'productid': 'HBV00000D1PZK', 'brand': 'FORA', 'category': 'KAHVALTILIK VE SÜT', 'subcategory': 'ZEYTIN', 'name': 'FORA ZEYTIN GEMLIK ÇEŞIDI DOĞAL SIYAH ZEYTIN 500 GR'}, {'productid': 'HBV00000D1PZO', 'brand': 'FORA', 'category': 'KAHVALTILIK VE SÜT', 'subcategory': 'ZEYTIN', 'name': 'FORA ZEYTIN AZ TUZLU SIYAH ZEYTIN 800 GR (L /231-260)'}, {'productid': 'ZYBICN9310823', 'brand': 'FORA', 'category': 'KAHVALTILIK VE SÜT', 'subcategory': 'ZEYTIN', 'name': 'FORA ZEYTIN AZ TUZLU SIYAH ZEYTIN 800 GR (S / 291-320)'}, {'productid': 'HBV00000NE1WZ', 'brand': 'FORA', 'category': 'KAHVALTILIK VE SÜT', 'subcategory': 'ZEYTIN', 'name': 'FORA GEMLIK KURU SELE SIYAH ZEYTIN 250 GR'}, {'productid': 'HBV00000AVVTL', 'brand': 'FORA'

In [ ]:
app = Flask(__name__)
CORS(app) 
        
@app.route('/events', methods=['GET'])
def Recommended_Products():

        SessionId =  request.args.get('sessionid')
        CardList = FindProductsWithRelatedSessionId(SessionId,Events)
        RecommendData = RecommendedProducts(cardsitem, Meta, c, t)
        recommendation_dict=RecommendData.to_dict("records")
        
        return jsonify(recommendation_dict)

app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.166:5000/ (Press CTRL+C to quit)
192.168.1.166 - - [01/Jun/2022 23:46:45] "GET /events?sessionid=f66b1455-d46c-45f5-b1c9-8af1d2fb12f3 HTTP/1.1" 200 -


['HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF']
[(253, 0.5371105831491507), (341, 0.5890969625071492), (523, 0.7691529366250116), (643, 0.7688705242838757), (835, 0.5634414865778319), (969, 0.5371105831491507), (2077, 0.6647009616560932), (2728, 0.5767100799945043), (6173, 0.6974603618736324), (6350, 0.664531135506103)]


192.168.1.166 - - [01/Jun/2022 23:46:49] "GET / HTTP/1.1" 404 -
192.168.1.166 - - [01/Jun/2022 23:47:00] "GET /events?sessionid=f66b1455-d46c-45f5-b1c9-8af1d2fb12f3 HTTP/1.1" 200 -


['HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF', 'HBV00000UIYEF']
[(253, 0.5371105831491507), (341, 0.5890969625071492), (523, 0.7691529366250116), (643, 0.7688705242838757), (835, 0.5634414865778319), (969, 0.5371105831491507), (2077, 0.6647009616560932), (2728, 0.5767100799945043), (6173, 0.6974603618736324), (6350, 0.664531135506103)]
